In [41]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Union
from datetime import datetime

# pydantic

Pydantic is a data validation library that uses Python type annotations. Its main purpose is to enforce type hints at runtime and provide data validation.

In [42]:
# Example

class User(BaseModel):
    name: str
    age: int
    email: str
    interests: List[str] = []  # Default value
    bio: Optional[str] = None  # Optional field
    metrics: Dict[str, float]
    zipcode: Union[str, int]   # Can be any of these types


# Creating an instance
user = User(
    name="Alice",
    age=30,
    email="alice@example.com",
    interests=["python", "data science"],
    metrics = {"age": 30},
    zipcode="12345"
)

## `Field()`

The `Field()` function is used to provide additional validation and metadata for model fields.

### Making a `Field()` Required

Here are all the ways to make a field required in Pydantic:

In [43]:
class Example(BaseModel):
    # Required - explicit with ...
    name: str = Field(...)

    # Required - no default specified
    email: str = Field()

    # Required - just type annotation
    age: int

    # All these fields are required and must be provided
    timestamp: datetime
    user_id: str = Field()
    status: str = Field(...)

### Making a `Field()` Optional 

* `default=` means optional with a default value. If not provided, it uses the default.
* `default_factory=` is similar to `default=` but calls a function to generate the default value
* To make a field truly OPTIONAL (can be `None`), use `Optional[]` from typing:
* Use `Optional[]` with `Field(default=None)` to add additional validation or metadata even though the field can be `None`.

If you don't specify a default value (either through `default=`, `default_factory=`, or `Optional[]`), then the field is required.

In [44]:
class User(BaseModel):
    # Optional with default value
    name: str = Field(default="Anonymous")

    # Default factory (calls function to generate default)
    # i.e. Optional with dynamic default value
    created_at: datetime = Field(default_factory=datetime.now)

    # Optional that can be None, simple version
    bio: Optional[str] = None

    # Optional that can be None, with additional validation
    age: Optional[int] = Field(
        default=None,
        ge=0,   # If age is provided, must be >= 0
        le=120, # If age is provided, must be < 120
        description="User's age (optional)"
    )

# All these are valid:
user1 = User()  # Uses all defaults
user2 = User(name="Alice", bio="Hello!", age=25)  # Provides values
user3 = User(bio=None, age=None)  # Explicitly sets to None

### Metadata and Documentation

In [51]:
class Example(BaseModel):

    description: str = Field(
        default="",
        title="Item Description",           # used in JSON schema
        description="Detailed description", # documentation
        examples=["Example text"]           # example values
    )

### Constraints

In [46]:
class Example(BaseModel):
    
    # Constraints for Numbers
    score: float = Field(
        gt=0,           # greater than
        lt=100,         # less than
        ge=0,           # greater than or equal
        le=100,         # less than or equal
        multiple_of=0.5 # must be multiple of this number
    )
    
    # String Constraints
    username: str = Field(
        min_length=3,    # minimum length
        max_length=50,   # maximum length
        pattern="^[a-zA-Z0-9_]*$"  # must match this pattern
    )
    
    # Sequence Constraints (Lists, Sets, etc)
    tags: List[str] = Field(
        min_items=1,     # minimum number of items
        max_items=10,    # maximum number of items
    )
    
    # Constrained Values
    status: str = Field(
        default="active",
        choices=["active", "inactive", "pending"]  # allowed values
    )

## Type Validation

In [47]:
# This will raise a validation error
try:
    user = User(name="Bob", age="not a number", email="bob@example.com")
except ValueError as e:
    print(f"Validation error: {e}")

Validation error: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not a number', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


## Data Coercion

In [48]:
class Event(BaseModel):
    timestamp: datetime
    
# Pydantic will automatically convert string to datetime
event = Event(timestamp="2024-01-13T10:30:00")
print(event.timestamp)  # Returns datetime object

2024-01-13 10:30:00


## Nested Models

In [49]:
class Address(BaseModel):
    street: str
    city: str
    country: str

class Customer(BaseModel):
    name: str
    address: Address
    
# Using nested models
customer = Customer(
    name="Alice",
    address={
        "street": "123 Main St",
        "city": "Boston",
        "country": "USA"
    }
)

## Data Export

In [53]:
# Convert to dictionary
user_dict = user.model_dump()
user_dict

{'name': 'Alice',
 'age': 30,
 'email': 'alice@example.com',
 'interests': ['python', 'data science'],
 'bio': None,
 'metrics': {'age': 30.0},
 'zipcode': '12345'}

In [54]:
# Convert to JSON
user_json = user.model_dump_json()
user_json

'{"name":"Alice","age":30,"email":"alice@example.com","interests":["python","data science"],"bio":null,"metrics":{"age":30.0},"zipcode":"12345"}'

In [55]:
# Exclude certain fields
user_dict = user.model_dump(exclude={'password'})
user_dict

{'name': 'Alice',
 'age': 30,
 'email': 'alice@example.com',
 'interests': ['python', 'data science'],
 'bio': None,
 'metrics': {'age': 30.0},
 'zipcode': '12345'}

# Enum

This code defines an enumeration (Enum) class called `Weapon` that inherits from both `str` and `Enum`. 

In the context of Pydantic, this is commonly used to define a set of valid string values that a field can accept.
1. The class inherits from `str` to make the enum values behave like strings
2. It also inherits from `Enum` to create an enumeration where each value is a constant
3. Each class attribute defines a valid enum member where:
- The name on the left (e.g., `sword`) is the enum member name
- The string on the right (e.g., "sword") is the value associated with that member

In [ ]:
from enum import Enum

class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"

class Character(BaseModel):
    name: str
    weapon: Weapon

# This works fine because "sword" is a valid weapon
character = Character(name="Aragorn", weapon="sword")

# This would raise a validation error because "hammer" isn't defined
# character = Character(name="Thor", weapon="hammer")

## Dyanmic Model Creation

There are some occasions where it is desirable to create a model using runtime information to specify the fields. For this Pydantic provides the `create_model` function to allow models to be created on the fly:

In [ ]:
from pydantic import BaseModel, create_model

DynamicFoobarModel = create_model(
    'DynamicFoobarModel', foo=(str, ...), bar=(int, 123)
)


class StaticFoobarModel(BaseModel):
    foo: str
    bar: int = 123

Here `StaticFoobarModel` and `DynamicFoobarModel` are identical.

Fields are defined by one of the following tuple forms:

- `(<type>, <default value>)`
- `(<type>, Field(...))`
- `typing.Annotated[<type>, Field(...)]`

Using a `Field(...)` call as the second argument in the tuple (the default value) allows for more advanced field configuration. Thus, the following are analogous:

In [ ]:
from pydantic import BaseModel, Field, create_model

DynamicModel = create_model(
    'DynamicModel',
    foo=(str, Field(description='foo description', alias='FOO')),
)


class StaticModel(BaseModel):
    foo: str = Field(description='foo description', alias='FOO')